In [4]:
# !python -m pip install emoji --upgrade

  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49721 sha256=9cd96555cd3d6a03e39369b00b9deaa63abf0270852c953dc50c1a7999ca4125
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\4e\bf\6b\2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


In [1]:
import json
from main import preprocess, make_meta
from tokenization import tokenize
from lemmatization import lemmatize

In [16]:
meta = make_meta()
len(meta)

192

Эксперименты (см. `evaluation`) показали, что наилучшим препроцессингом оказался следующий:

In [20]:
meta[81]

{'punctuation_deletion': 'no',
 'ner_processing': 'replace',
 'lemmatization': 'yes',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'yes'}

### Пример использования на 1 вхождении

In [3]:
test = '[id00000|имя], какая у вас ох*енная собака! 😍💤😥'
tok = tokenize(test)

for i, params in meta.items():
    print(preprocess(tok, params=params))

username , какая у вас охёенная собака ! 😍 💤 😥
username , какая у вас vulgar собака ! 😍 💤 😥
username , какая у вас охёенная собака ! 
username , какая у вас vulgar собака !
username , какая у вас охёенная собака ! emoji emoji emoji
username , какая у вас vulgar собака ! emoji emoji emoji
username , какая у вас охёенная собака ! pos_emoji neutral_emoji neg_emoji
username , какая у вас vulgar собака ! pos_emoji neutral_emoji neg_emoji
username , охёенная собака ! 😍 💤 😥
username , vulgar собака ! 😍 💤 😥
username , охёенная собака ! 
username , vulgar собака !
username , охёенная собака ! emoji emoji emoji
username , vulgar собака ! emoji emoji emoji
username , охёенная собака ! pos_emoji neutral_emoji neg_emoji
username , vulgar собака ! pos_emoji neutral_emoji neg_emoji
username , какой у вы охёенный собака ! 😍 💤 😥
username , какой у вы vulgar собака ! 😍 💤 😥
username , какой у вы охёенный собака ! 
username , какой у вы vulgar собака !
username , какой у вы охёенный собака ! emoji emoji e

## Пример использования на всех данных

In [4]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [5]:
from merge_df import table_to_dataframe

In [6]:
filepath = '../Data/data.xlsx'
raw_df = table_to_dataframe(filepath)
raw_df.dropna(axis=0, how='any', inplace=True)
raw_df.drop(columns=['index'], inplace=True)
len(raw_df)

4669

In [7]:
df = raw_df[raw_df['разметка'] != 'skip'].copy()
df['разметка'].value_counts()

negative    491
positive    410
neutral     357
speech      213
Name: разметка, dtype: int64

In [8]:
print('итого вхождений:', len(df))

итого вхождений: 1471


In [9]:
df['tokenized'] = df['текст'].progress_apply(tokenize)
df['lemmatized'] = df['tokenized'].progress_apply(lemmatize)

In [10]:
df.head()

,разметка,текст,контекст,community_id,community_type,tokenized,lemmatized
0,speech,Спасибо!,post,1,nontoxic,спасибо !,спасибо !
1,speech,вау спасибо большое!!,post,1,nontoxic,вау спасибо большое ! !,вау спасибо большой ! !
2,speech,Супер! Благодарю🙏,post,1,nontoxic,супер ! благодарю 🙏,супер ! благодарить 🙏
3,speech,"офигеть, спасибо!",post,1,nontoxic,"офигеть , спасибо !","офигеть , спасибо !"
4,speech,спасибо за проделанную работу 💙,post,1,nontoxic,спасибо за проделанную работу 💙,спасибо за проделать работа 💙


Для ускорения я сначала все лемматизирую. В итоговом датасете порядок выполнения операций как и написан в `main.py`

In [11]:
for i, params in tqdm(meta.items()):
    if params['lemmatization'] == 'no':
        df[i] = [preprocess(tok, params=params) for tok in df['tokenized']]
    if params['lemmatization'] == 'yes':
        params['lemmatization'] = 'no'
        df[i] = [preprocess(tok, params=params) for tok in df['lemmatized']]

In [13]:
df.sample(5)

,разметка,текст,контекст,community_id,community_type,tokenized,lemmatized,0,1,2,...,182,183,184,185,186,187,188,189,190,191
4790,negative,хуёвый ты друг,я обычно такому сходу в ебало даю и дальше про...,3,toxic,хуевый ты друг,хуевыя ты друг,хуевый ты друг,vulgar ты друг,хуевый ты друг,...,хуевыя ты друг,хуевыя ты друг,хуевыя друг,хуевыя друг,хуевыя друг,хуевыя друг,хуевыя друг,хуевыя друг,хуевыя друг,хуевыя друг
4619,negative,"[id60624256|Алексей], а я тебе советую меньше ...","[id24019984|Иван], я бы тебе посоветовал вмест...",3,toxic,"username , а я тебе советую меньше леше яйца л...","username , а я ты советовать маленький лёша яй...","username , а я тебе советую меньше леше яйца л...","username , а я тебе советую меньше леше яйца л...","username , а я тебе советую меньше леше яйца л...",...,username а я ты советовать маленький лёша яйцо...,username а я ты советовать маленький vulgar яй...,username советовать маленький лёша яйцо лизать,username советовать маленький vulgar яйцо лизать,username советовать маленький лёша яйцо лизать,username советовать маленький vulgar яйцо лизать,username советовать маленький лёша яйцо лизать,username советовать маленький vulgar яйцо лизать,username советовать маленький лёша яйцо лизать,username советовать маленький vulgar яйцо лизать
3444,negative,не матерись. тебя бомжи что ли воспитывали?,"[id96091477|Игорь], ты выдумка, бот, пошел нах...",3,toxic,не матерись . тебя бомжи что ли воспитывали ?,не материться . ты бомж что ли воспитывать ?,не матерись . тебя бомжи что ли воспитывали ?,не матерись . тебя бомжи что ли воспитывали ?,не матерись . тебя бомжи что ли воспитывали ?,...,не материться ты бомж что ли воспитывать,не материться ты бомж что ли воспитывать,материться бомж воспитывать,материться бомж воспитывать,материться бомж воспитывать,материться бомж воспитывать,материться бомж воспитывать,материться бомж воспитывать,материться бомж воспитывать,материться бомж воспитывать
1029,neutral,Он на Поперечного чем-то похож,post,2,nontoxic,он на поперечного чем-то похож,он на поперечный что-то похожий,он на поперечного чем-то похож,он на поперечного чем-то похож,он на поперечного чем-то похож,...,он на поперечный что-то похожий,он на поперечный что-то похожий,поперечный похожий,поперечный похожий,поперечный похожий,поперечный похожий,поперечный похожий,поперечный похожий,поперечный похожий,поперечный похожий
7004,positive,"[id43340308|Иван], 😆😆😆😆","[id160622183|Альгиз], так скольки раз там пида...",4,toxic,"username , 😆 😆 😆 😆","username , 😆 😆 😆 😆","username , 😆 😆 😆 😆","username , 😆 😆 😆 😆","username ,",...,username pos_emoji pos_emoji pos_emoji pos_emoji,username pos_emoji pos_emoji pos_emoji pos_emoji,username 😆 😆 😆 😆,username 😆 😆 😆 😆,username,username,username emoji emoji emoji emoji,username emoji emoji emoji emoji,username pos_emoji pos_emoji pos_emoji pos_emoji,username pos_emoji pos_emoji pos_emoji pos_emoji


In [19]:
df.to_csv('../Data/data_preprocessed.tsv', sep='\t')

### Внимание: в датасете моугт быть пустые ячейки!

Откуда? Они возникают в вариантах препроцессинга, где используется удаление стоп-слов и/или удаление эмоджи (например, когда комментарий изначально состоял только из эмоджи).

Как с этим бороться:
```python
df.fillna('none',  inplace=True)
assert data.isnull().values.any() == False
```

Если нужно поменять опции то допишите дополнительные циклы main.py